# 1. Análise Exploratória de Dados - Evasão Escolar

Limpar e explorar os dados do Censo Escolar para entender melhor o fenômeno da evasão.

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [32]:
FILE_PATH = '../data/raw/microdados_ed_basica_2024.csv'

# Carregar os dados. Pode demorar alguns minutos.
df = pd.read_csv(FILE_PATH, sep=';', encoding='latin-1', low_memory=False, dtype=str)

print("Dados carregados com sucesso!")
print(f"Shape do dataframe: {df.shape}")

Dados carregados com sucesso!
Shape do dataframe: (215545, 426)


In [18]:
# Este código vai listar todas as colunas do seu DataFrame
lista_de_colunas = df.columns.tolist()

# E este vai procurar por colunas que tenham a palavra 'SITUACAO' no nome
colunas_provaveis = [nome_da_coluna for nome_da_coluna in lista_de_colunas if 'ANO' in nome_da_coluna]

print("--- Possíveis colunas de 'Situação do Aluno' encontradas ---")
print(colunas_provaveis)

--- Possíveis colunas de 'Situação do Aluno' encontradas ---
['NU_ANO_CENSO', 'DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'IN_AGUA_POCO_ARTESIANO']


In [33]:
# Selecionar colunas relevantes para evitar uso excessivo de memória
colunas_relevantes = [
    #'NU_IDADE',
    #'TP_SEXO',
    #'TP_COR_RACA',
    #'TP_ZONA_RESIDENCIAL',
    #'TP_DEPENDENCIA', # Federal, Estadual, Municipal, Privada
    'DT_ANO_LETIVO_TERMINO' # Situação do aluno no final do ano anterior
]

# 1 se o aluno deixou de frequentar, 0 caso contrário
df['EVADIU'] = np.where(df['DT_ANO_LETIVO_TERMINO'] == 4, 1, 0)


df_limpo = df[colunas_relevantes + ['EVADIU']].copy()

for col in df_limpo.columns:
    if df_limpo[col].isnull().any():
        moda = df_limpo[col].mode()[0]
        df_limpo[col].fillna(moda, inplace=True)

print("DataFrame limpo e variável alvo criada.")
print(df_limpo.head())

DataFrame limpo e variável alvo criada.
  DT_ANO_LETIVO_TERMINO  EVADIU
0    13DEC2024:00:00:00       0
1    19DEC2024:00:00:00       0
2    20DEC2024:00:00:00       0
3    14DEC2024:00:00:00       0
4    14DEC2024:00:00:00       0


C:\Users\guilh\AppData\Local\Temp\ipykernel_11648\3296509653.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_limpo[col].fillna(moda, inplace=True)


In [34]:
# Taxa geral de evasão
print(f"Taxa de Evasão na amostra: {df_limpo['EVADIU'].mean() * 100:.2f}%")

# Evasão por Sexo (1=Masc, 2=Fem)
sns.countplot(x='TP_SEXO', hue='EVADIU', data=df_limpo)
plt.title('Evasão por Sexo')
plt.show()

# Evasão por Idade
sns.boxplot(x='EVADIU', y='NU_IDADE', data=df_limpo)
plt.title('Distribuição de Idade por Situação de Evasão')
plt.show()

Taxa de Evasão na amostra: 0.00%


ValueError: Could not interpret value `TP_SEXO` for `x`. An entry with this name does not appear in `data`.

In [35]:
# Salvar o dataframe limpo para ser usado no próximo notebook
df_limpo.to_csv('../data/processed/dados_limpos_para_modelo.csv', index=False)
print("Dados processados salvos com sucesso!")

Dados processados salvos com sucesso!
